In [78]:
import numpy as np
from scipy.sparse.linalg import svds
from sklearn import datasets

In [102]:
def e(i, d):
    ei = np.zeros(d)
    ei[i] = 1
    return ei

def KWSA(F, w, c, d):
    """ 
    Kiefer-Wolfowitz stochastic approximation
    for gradient estimation 
    
    INPUT:
    - F: objective function
    - w: current weight
    - d: dimension
    - c: costant
    
    """

    g = np.zeros(d)
    for i in range(d):
        g[i] = (F(w + c * e(i, d)) - F(w))/c
    return g

def detZFW(F, L, d, w0, r=1, T=100, eps=1e-16):
    """
    INPUT
    - F: loss function
    - L: Lip constant
    - d: dimension
    - w0: starting point
    - r: aius of the ball
    - T: mawiteration
    - eps: tolerance
    """

    gamma = lambda t: 2/(t+1)
    c = lambda t: L*gamma(t)/d
    w = w0
    partial = 0
    for t in range(T):
        # comupute the gradient approx
        gt = KWSA(F, w, c(t), d)
        print('g:', gt)
        # comupute the linear problem solution ON L1 BALL
        ei = e(np.argmin(np.abs(gt)), d)
        print('ei:', ei)
        v = np.sign(gt) * ei
        print('np.sign(g):',np.sign(gt))
        print('v:', v)
        # compute step 
        w_pred = w
        w = (1 - gamma(t)) * w + gamma(t) * v
        print('w:', w)
        print('c:',c(t))
        print()
        partial += w
        # check stopping condition
        if np.abs(F(w) - F(w_pred)) < eps:
            break
    return F(w_pred), F(w), w, partial/T, t

## Stochastic lasso regression

In [103]:
# load data
X, y = datasets.load_svmlight_file("../Data/covtype.libsvm.binary.scale.bz2")

In [104]:
# space dimension
d = X.shape[1]
d

54

In [105]:
# label diminsione (num. data)
y.shape

(581012,)

In [112]:
# define the objective function
F = lambda w: 0.5 * sum(np.power(y - X @ w, 2))

In [113]:
# initialize prarameters for the algorithm

# stating point - TODO: randomize
w0 = np.random.rand(d)
w0 = w0/sum(w0)
print(w0)
print(F(w0))

# Lipschitz constant computation
L = 2/X.shape[0] * np.linalg.norm( X.T @ X )
print(L/52)


[0.02872196 0.01963129 0.02856883 0.02610322 0.02698978 0.02528784
 0.00911648 0.0079951  0.02567307 0.02216705 0.02799944 0.02030039
 0.00663008 0.00529333 0.01790368 0.00172445 0.00037278 0.03136401
 0.01427368 0.02597213 0.03208377 0.00444574 0.00224823 0.01379581
 0.01622351 0.02814218 0.02465321 0.02357909 0.01677044 0.00261315
 0.01614237 0.01480502 0.02510866 0.03131734 0.02626207 0.01484406
 0.0046146  0.03135223 0.02443178 0.02795423 0.0136868  0.01406522
 0.03018458 0.0105576  0.02606947 0.01357573 0.02280252 0.01094493
 0.02438218 0.01522351 0.02750041 0.01652063 0.00691639 0.01409394]
612691.7334076834
  (0, 0)	0.03793059317652083
  (1, 0)	0.033876597525975514
  (2, 0)	0.023184040304061643
  (3, 0)	0.02250906399577856
  (4, 0)	0.027009523710906126
  (5, 0)	0.029786252243662988
  (6, 0)	0.04630138362469912
  (7, 0)	0.04766076148945145
  (8, 0)	0.03820059748971479
  (9, 0)	0.02697122122720304
  (10, 0)	0.03471936504807004
  (11, 0)	0.01098337089260513
  (12, 0)	0.033497068920

In [114]:
X.T @ X 

<54x54 sparse matrix of type '<class 'numpy.float64'>'
	with 1174 stored elements in Compressed Sparse Column format>

In [101]:
fpred, f, w, mean, t = detZFW(F, L, d, w0)

print('F(w_pred) =', fpred, '\n',
      'F(w) =', f, '\n',
      'w =', w, '\n',
      'mean w =', mean, '\n',
      't =', t)

g: [2.11189373e+09 1.85645564e+09 3.83099310e+08 3.94944153e+08
 5.63096684e+08 1.02795217e+09 4.63946990e+09 5.10012231e+09
 2.20981274e+09 7.21937310e+08 2.93851049e+09 3.36725328e+08
 2.85479075e+09 4.16553424e+08 3.41533484e+07 8.47910345e+07
 5.43445386e+07 1.39675301e+08 1.79950187e+07 7.40864434e+07
 1.18303839e+06 2.01684506e+06 1.29234137e+07 3.67709927e+08
 1.39827240e+08 3.37686924e+08 1.96400199e+08 6.74955773e+06
 3.38039588e+04 3.20558346e+07 3.85582987e+07 2.13962966e+07
 4.53073446e+07 1.04325601e+08 9.44260071e+06 3.76041296e+08
 6.50731777e+08 2.39751651e+08 5.34069247e+06 2.91707985e+07
 1.22366404e+07 1.06586863e+07 1.29853367e+09 3.39936271e+08
 2.89192115e+08 5.91756911e+08 5.08771145e+08 1.81513788e+07
 2.13078722e+07 1.34085687e+06 3.35786787e+06 1.75476324e+08
 1.55566402e+08 9.85950324e+07]
ei: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
np

g: [9.04812116e+08 7.95398209e+08 1.64074991e+08 1.69159462e+08
 2.41178826e+08 4.40376092e+08 1.98790617e+09 2.18530505e+09
 9.46768182e+08 3.09252958e+08 1.25911641e+09 1.44286991e+08
 1.22325205e+09 1.78498257e+08 1.46352865e+07 3.63338720e+07
 2.32868351e+07 5.98512735e+07 7.71116886e+06 3.17467101e+07
 5.06891201e+05 8.64172028e+05 5.53734171e+06 1.57563601e+08
 5.99130478e+07 1.44688752e+08 8.41529656e+07 2.89229304e+06
 1.44883965e+04 1.37354415e+07 1.65222939e+07 9.16772692e+06
 1.94140553e+07 4.47022073e+07 4.04629225e+06 1.61135311e+08
 2.78836882e+08 1.02731882e+08 2.28837632e+06 1.24989062e+07
 5.24333883e+06 4.56690459e+06 5.56402679e+08 1.45656687e+08
 1.23915977e+08 2.53560332e+08 2.18002354e+08 7.77733993e+06
 9.13075049e+06 5.74553896e+05 1.43890312e+06 7.51940392e+07
 6.66624200e+07 4.22493569e+07]
ei: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
np

g: [5.75617585e+08 5.06026866e+08 1.04344711e+08 1.07585215e+08
 1.53387937e+08 2.80133842e+08 1.26476664e+09 1.39037006e+09
 6.02311349e+08 1.96707415e+08 8.01107193e+08 9.18047681e+07
 7.78294753e+08 1.13575158e+08 9.31225465e+06 2.31184888e+07
 1.48166945e+07 3.80814491e+07 4.90652285e+06 2.01997101e+07
 3.22490471e+05 5.49812542e+05 3.52298474e+06 1.00251864e+08
 3.81186361e+07 9.20537589e+07 5.35406711e+07 1.84033050e+06
 9.21994372e+03 8.73905867e+06 1.05125841e+07 5.83270968e+06
 1.23523821e+07 2.84415710e+07 2.57459833e+06 1.02525703e+08
 1.77412918e+08 6.53635057e+07 1.45593900e+06 7.95210943e+06
 3.33610806e+06 2.90554099e+06 3.54006276e+08 9.26722130e+07
 7.88414136e+07 1.61326569e+08 1.38703028e+08 4.94810544e+06
 5.80977960e+06 3.65566268e+05 9.15556542e+05 4.78447768e+07
 4.24163447e+07 2.68826257e+07]
ei: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
np

g: [4.21993902e+08 3.70987266e+08 7.64707438e+07 7.88507199e+07
 1.12419077e+08 2.05354387e+08 9.27302167e+08 1.01940109e+09
 4.41565272e+08 1.44186375e+08 5.87369891e+08 6.73131073e+07
 5.70648367e+08 8.32777585e+07 6.82817656e+06 1.69513194e+07
 1.08639687e+07 2.79222143e+07 3.59768987e+06 1.48111192e+07
 2.36436935e+05 4.03111716e+05 2.58295268e+06 7.35064278e+07
 2.79479273e+07 6.74908014e+07 3.92549602e+07 1.34941549e+06
 6.76130652e+03 6.40741733e+06 7.70805784e+06 4.27637044e+06
 9.05694056e+06 2.08532872e+07 1.88780904e+06 7.51746037e+07
 1.30081821e+08 4.79249597e+07 1.06746881e+06 5.83027469e+06
 2.44606853e+06 2.13023940e+06 2.59554756e+08 6.79461642e+07
 5.78066511e+07 1.18284222e+08 1.01696732e+08 3.62779822e+06
 4.25999600e+06 2.68038893e+05 6.71328476e+05 3.50818081e+07
 3.11015306e+07 1.97114968e+07]
ei: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
np

g: [3.33053968e+08 2.92806522e+08 6.03332194e+07 6.22149927e+07
 8.87003109e+07 1.62061074e+08 7.31928138e+08 8.04629727e+08
 3.48501851e+08 1.13779504e+08 4.63627314e+08 5.31337341e+07
 4.50432114e+08 6.57371691e+07 5.39002683e+06 1.33808549e+07
 8.57554981e+06 2.20405556e+07 2.83994486e+06 1.16914106e+07
 1.86616497e+05 3.18179716e+05 2.03872384e+06 5.80222366e+07
 2.20596258e+07 5.32701504e+07 3.09842913e+07 1.06520172e+06
 5.33787939e+03 5.05752058e+06 6.08438581e+06 3.37533239e+06
 7.14905467e+06 1.64600730e+07 1.49019444e+06 5.93397674e+07
 1.02679626e+08 3.78289658e+07 8.42565146e+05 4.60184486e+06
 1.93078282e+06 1.68138089e+06 2.04872325e+08 5.36310917e+07
 4.56286380e+07 9.33649844e+07 8.02720469e+07 2.86341031e+06
 3.36275347e+06 2.11575716e+05 5.29933353e+05 2.76927254e+07
 2.45508533e+07 1.55597933e+07]
ei: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
np

g: [2.75049695e+08 2.41819107e+08 4.98087590e+07 5.13656167e+07
 7.32315667e+07 1.33826324e+08 6.04510340e+08 6.64561498e+08
 2.87808348e+08 9.39489509e+07 3.82925658e+08 4.38863200e+07
 3.72030236e+08 5.42976577e+07 4.45210334e+06 1.10522918e+07
 7.08310320e+06 1.82046924e+07 2.34576346e+06 9.65681867e+06
 1.54124916e+05 2.62789300e+05 1.68379209e+06 4.79238540e+07
 1.82194304e+07 4.39958156e+07 2.55903787e+07 8.79844979e+05
 4.40955546e+03 4.17715342e+06 5.02546963e+06 2.78769904e+06
 5.90478170e+06 1.35949343e+07 1.23088066e+06 4.90127039e+07
 8.48086357e+07 3.12446241e+07 6.95888883e+05 3.80069524e+06
 1.59472703e+06 1.38864719e+06 1.69209880e+08 4.42951778e+07
 3.76864580e+07 7.71133134e+07 6.62994303e+07 2.36489662e+06
 2.77759550e+06 1.74751918e+05 4.37719166e+05 2.28737599e+07
 2.02786740e+07 1.28521614e+07]
ei: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
np

g: [2.34231887e+08 2.05939086e+08 4.24026624e+07 4.37308754e+07
 6.23461611e+07 1.13957434e+08 5.14845984e+08 5.65994987e+08
 2.45098119e+08 7.99941244e+07 3.26135614e+08 3.73788818e+07
 3.16858555e+08 4.62476326e+07 3.79208322e+06 9.41367357e+06
 6.03286320e+06 1.55053818e+07 1.99800624e+06 8.22506910e+06
 1.31260475e+05 2.23810868e+05 1.43402533e+06 4.08175861e+07
 1.55170712e+07 3.74694331e+07 2.17946632e+07 7.49408780e+05
 3.75628964e+03 3.55763592e+06 4.28030655e+06 2.37417934e+06
 5.02918240e+06 1.15787260e+07 1.04840064e+06 4.17455127e+07
 7.22327562e+07 2.66111993e+07 5.92672271e+05 3.23692341e+06
 1.35824337e+06 1.18264944e+06 1.44114090e+08 3.77254618e+07
 3.20975176e+07 6.56769547e+07 5.64668500e+07 2.01409075e+06
 2.36581777e+06 1.48838882e+05 3.72827713e+05 1.94826367e+07
 1.72723263e+07 1.09467912e+07]
ei: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
np

g: [2.03947714e+08 1.79318430e+08 3.69078192e+07 3.80663924e+07
 5.42698960e+07 9.92160036e+07 4.48320827e+08 4.92865007e+08
 2.13409892e+08 6.96405468e+07 2.84001071e+08 3.25507832e+07
 2.75924733e+08 4.02750342e+07 3.30239092e+06 8.19792473e+06
 5.25365299e+06 1.35026677e+07 1.73999284e+06 7.16280343e+06
 1.14296538e+05 1.94891391e+05 1.24871453e+06 3.55451944e+07
 1.35120953e+07 3.26272790e+07 1.89784877e+07 6.52633549e+05
 3.27160815e+03 3.09799396e+06 3.72744370e+06 2.06737444e+06
 4.37954430e+06 1.00828298e+07 9.13012249e+05 3.63537264e+07
 6.29022664e+07 2.31734976e+07 5.16092214e+05 2.81864114e+06
 1.18278778e+06 1.02981242e+06 1.25494635e+08 3.28511570e+07
 2.79508850e+07 5.71919156e+07 4.91717107e+07 1.75381547e+06
 2.06030530e+06 1.29613083e+05 3.24682446e+05 1.69666423e+07
 1.50418106e+07 9.53312966e+06]
ei: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
np

KeyboardInterrupt: 

In [71]:
F(mean)

714260.8225783304

In [45]:
np.sign([0.3, -0.4]) * np.array([0, 1])

array([ 0., -1.])